##### Dereferencing a pointer to variable beyond its lifetime is UB.

In [11]:
#include <stdio.h>

int* foo(int bar)
{
int baz = 6;
baz += bar;
return &baz; /* (&baz) copied to new memory location outside of foo. */
} /* (1) The lifetime of baz and bar end here as they have automatic storage * duration (local variables), thus the returned pointer is not valid! */

int main (void)
{
int* p;
p = foo(5); /* (2) this expression's behavior is undefined */
*p = *p - 6; /* (3) Undefined behaviour here */ return 0;
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp8c61zxo5.c:7:9: warning: address of stack memory associated with local variable 'baz' returned [-Wreturn-stack-address]
return &baz; /* (&baz) copied to new memory location outside of foo. */
        ^~~
1 warning generated.


##### Signed Integer Overflow

In [16]:
#include <stdio.h>
#include <limits.h>

int main(void)
{
printf("%d,%d\n",INT_MAX, INT_MIN);
printf("%d\n",INT_MAX+1);
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpum02km6y.c:7:22: warning: overflow in expression; result is -2147483648 with type 'int' [-Winteger-overflow]
printf("%d\n",INT_MAX+1);
                     ^
1 warning generated.


2147483647,-2147483648
-2147483648


##### Using uninitialized variable is UB. 

In [26]:
#include <stdio.h>

int main(void) {
    int i, counter;
    printf("%d\n", counter);
    for(i = 0; i < 10; ++i)
        counter += i;
printf("%d\n", counter);
return 0; }

32767
32812


##### Increment operator


In [27]:
#include <stdio.h>

int main(void)
{int i = 42;
i = i++; /* Assignment changes variable, post-increment as well */ 
int a = i++ + i--;
printf("%d",i);
}



/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpnheg4rb9.c:5:6: warning: multiple unsequenced modifications to 'i' [-Wunsequenced]
i = i++; /* Assignment changes variable, post-increment as well */ 
  ~  ^
/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpnheg4rb9.c:6:10: warning: multiple unsequenced modifications to 'i' [-Wunsequenced]
int a = i++ + i--;
         ^     ~~
2 warnings generated.


42

#### Conversion of `signed int` into `unsigned int`

In [28]:
#include <stdio.h>
int main(void)
{
	unsigned int a = 1000;
	signed int b = -1;
	
	if(a>b) puts("a is more than b");
	else puts("a is less than or equal to b");
	
	return 0;
}

a is less than or equal to b


Since 1000 is more than -1 you would expect the output to be `a is more than b`, however that will not be the case. Arithmetic operations between different integral types are performed within a common type defined by the so called usual arithmetic conversions (see the language specification, 6.3.1.8). In this case the "common type" is `unsigned int`, Because, as stated in Usual arithmetic conversions,

>714- *Otherwise, if the operand that has unsigned integer type has rank greater or equal to the rank of the type of the other operand, then the operand with signed integer type is converted to the type of the operand with unsigned integer type.*

This means that `int operand b` will get converted to `unsigned int` before the comparison. When -1 is converted to an `unsigned int` the result is the maximal possible `unsigned int` value, which is greater than 1000, meaning that `a > b` is false.


From the book, "C Programming - A Modern Approach" -

> When a signed operand is combined with an unsigned operand, the signed operand is converted to an unsigned value. The conversion involves adding or subtracting a multiple of n+1, where n is the largest representable value of unsigned type. This rule can cause obsucre programming errors. 
> Suppose that the `int` variable `i` has the value -10 and `unsigned int` variable `u` has the value 10. If we compare `i` and `u` using the < operator, we might expect the result 1 (True). Before the comparison, however, `i` is converted to `unsigned int`. Since a negative number can't be represented as an unsigned integer, the converted value won't be -10. Instead, the value 4,294,967,296 is added (assuming that 4,294,967,295 is the largest  `unsigned int` value), giving the converted value of 4,294,967,286. The comparison `i < u` will therefore produce 0. 

